## DSCB310 - Data Analytics und Business Intelligence - Projekt 2 - WS 2022/23 - Matrikelnummern 80055, 80173
#### Inhaltsverzeichnis des Notebooks
1. Einleitung
2. Import der benötigten Bibliotheken
3. Feature Engineering und Funktionen
4. Genereller Überblick über Produkte, Kategorien, Gänge und Counties
5. Untersuchung auf besondere Produkte
6. Fragen der Personalabteilung
    + Frage P.1: Untersuchung auf ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung: 24852, 24964, 2120
    + Frage P.2: Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?
    + Frage P.3: Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?
    + Frage P.4: Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?
7. Fragen der Disposition
    + Frage D.1: Prüfung auf auffällige Muster bei dem Wiederbestellverhalten des Produktes "Bag of Organic Bananas" (Produkt-ID: 13176)
    + Frage D.2: Spielen die Artikel aus dem Department „produce“ in allen Counties eine gleich große Rolle?
    + Frage D.3: Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte?
    + Räumliche Analyse der Bestellungsdaten für die Disposition
    + Spezifische Handlungsempfehlungen für die Disposition

## Einleitung
Im Rahmen der Prüfungsleistung soll, basierend auf dem Datensatz "shopping_carts.parquet", welcher Verkaufsdaten eines kalifornischen Online-Lebensmittel-Lieferanten darstellt, in beratender Funktion Schlüsse gezogen werden, um diesem eine bessere Durchdringung des Geschäftsumfeldes zu ermöglichen und die Fragen der Personalabteilung sowie der Distribution bestmöglich zu beantworten und diesen Handlungsempfehlungen zu geben.

## Import der benötigten Bibliotheken

In [ ]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score
from scipy.stats import chi2_contingency
from urllib.request import urlopen
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import json

## Feature Engineering und Funktionen

In [ ]:
df = pd.read_parquet('shopping_carts.parquet')

In [ ]:
# Berechnet die relative Trinkgeldhäufigkeit des Users bis zu der Bestellung
df_temp = df.groupby(['user_id', 'order_number'])['tip'].first().reset_index()
df_temp['tip_sum'] = df_temp.groupby(['user_id'])['tip'].cumsum()
df_temp['tip_percentage_untill_this_order'] = df_temp['tip_sum'].shift() / df_temp['order_number'].shift()

# Gibt es noch keine Bestellung, so ist die relative Häufigkeit 0
df_temp.loc[df_temp['order_number'] == 1,'tip_percentage_untill_this_order'] = 0
 
df_extended = df.merge(df_temp[['user_id', 'order_number', 'tip_percentage_untill_this_order']], on=['user_id', 'order_number'])

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_geojson = json.load(response)

california_counties_geo = [x for x in counties_geojson['features'] if x['properties']['STATE'] == '06']
counties_geojson['features'] = california_counties_geo

def plot_graph_of_california(df, color_attribute, title, color_label, color_descrete_sequence=px.colors.qualitative.Dark24, color_continuous_scale=px.colors.sequential.Plasma,opacity=0.3, hover_data = [],discrete_color_map={}, return_figure=False, number=None):
    fig = px.choropleth_mapbox(df,
                                geojson=counties_geojson,
                                featureidkey= "properties.NAME",
                                locations='county',
                                color=color_attribute,
                                title=title,
                                mapbox_style="carto-positron",
                                zoom=4.3,
                                center = {"lat": 37.278259, "lon": -119.417931},
                                opacity=opacity,
                                color_discrete_sequence=color_descrete_sequence,
                                color_continuous_scale=color_continuous_scale,
                                color_discrete_map= discrete_color_map if len(discrete_color_map) > 0 else None,
                                labels={color_attribute: color_label},
                                hover_data=hover_data)
    fig.update_layout(margin={"r":0,"l":0,"b":0})
    fig.add_annotation(
        showarrow=False,
        text=f'Abb. {number}' if number else '',
        font=dict(size=10),
        xanchor='left',
        yanchor='bottom',
        x=0.01,
        y=0.05,
        xref='paper',
        yref='paper'
    )
    return fig if return_figure else fig.show()

def print_evaluation(pipeline_or_model, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred):
    """ Ausgabe von Acc, Rec, Pre für Trainings- und Testset """

    accurary_train = accuracy_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)

    accurary_test = accuracy_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'ACC':>10} | {'REC':>14} | {'PRE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {accurary_train:10.5f} | {recall_train:14.5f} | {precision_train:10.5f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {accurary_test:10.5f} | {recall_test:14.5f} | {precision_test:10.5f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )

## Genereller Überblick über Produkte, Kategorien und Counties

In [ ]:
order_count_by_product_id = df_extended.groupby('product_id')['order_id'].count().reset_index().rename(columns={'order_id': 'order_count'}).sort_values('order_count', ascending=False).head(10)
order_count_by_product_id = order_count_by_product_id.merge(df_extended[['product_id', 'product_name']].drop_duplicates(), on='product_id')
fig = px.bar(order_count_by_product_id, x='product_name', y='order_count', title="Bestellhäufigkeit der Top 10 Produkte", text_auto='.2s')
fig.update_xaxes(title='Produkt')
fig.update_yaxes(title='Bestellhäufigkeit')
fig.add_annotation(
    showarrow=False,
    text='Abb. 1',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.5,
    xref='paper',
    yref='paper'
    )
fig.show()

In [ ]:
order_count_by_department_id = df_extended.groupby('department_id')['order_id'].count().reset_index().rename(columns={'order_id': 'order_count'}).sort_values('order_count', ascending=False)
order_count_by_department_id = order_count_by_department_id.merge(df_extended[['department_id', 'department']].drop_duplicates(), on='department_id')
fig = px.bar(order_count_by_department_id, x='department', y='order_count', title="Bestellhäufigkeit nach Abteilungen<br><sup>Sind in einer Bestellung N Produkte einer Kategorie, so wird zu der Bestellhäufigkeit N addiert</sup>", text_auto='.2s')
fig.update_xaxes(title='Abteilung')
fig.update_yaxes(title='Bestellhäufigkeit')
fig.add_annotation(
    showarrow=False,
    text='Abb. 2',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.3,
    xref='paper',
    yref='paper'
    )
fig.show()

# fig2 = px.sunburst(order_count_by_department_id, path=['department'], values='order_count', title='Bestellhäufigkeit nach Abteilung')
# fig2.show()

In [ ]:
product_count_by_department = df_extended.groupby('department_id')['product_id'].nunique().reset_index().rename(columns={'product_id': 'product_count'}).sort_values('product_count', ascending=False)
product_count_by_department = product_count_by_department.merge(df_extended[['department_id', 'department']].drop_duplicates(), on='department_id')
fig = px.bar(product_count_by_department, x='department', y='product_count', title="Umfang des Sortiments nach Abteilung", text_auto='.2s')
fig.update_xaxes(title='Abteilung')
fig.update_yaxes(title='Anzahl der Produkte')
fig.add_annotation(
    showarrow=False,
    text='Abb. 24',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.3,
    xref='paper',
    yref='paper'
    )
fig.show()

In [ ]:
order_count_by_aisle_id = df_extended.groupby('aisle_id')['order_id'].count().reset_index().rename(columns={'order_id': 'order_count'}).sort_values('order_count', ascending=False).head(10)
order_count_by_aisle_id = order_count_by_aisle_id.merge(df_extended[['aisle_id', 'aisle']].drop_duplicates(), on='aisle_id')
fig = px.bar(order_count_by_aisle_id, x='aisle', y='order_count', title="Bestellhäufigkeit der Top 10 Gänge<br><sup>Sind in einer Bestellung N Produkte eines Ganges, so wird zu der Bestellhäufigkeit N addiert</sup>", text_auto='.2s')
fig.update_xaxes(title='Gang')
fig.update_yaxes(title='Bestellhäufigkeit')
fig.add_annotation(
    showarrow=False,
    text='Abb. 3',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.5,
    xref='paper',
    yref='paper'
    )
fig.show()

In [ ]:
order_count_by_county = df_extended.groupby('county')['order_id'].nunique().reset_index().rename(columns={'order_id': 'order_count'}).sort_values('order_count', ascending=False)
fig = px.bar(order_count_by_county, x='county', y='order_count', title="Anzahl an Bestellungen nach County", text_auto='.2s')
fig.update_xaxes(title='County')
fig.update_yaxes(title='Bestellhäufigkeit')
fig.add_annotation(
    showarrow=False,
    text='Abb. 4',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.5,
    xref='paper',
    yref='paper'
    )
fig.show()
order_count_by_county_log = order_count_by_county.copy()
order_count_by_county_log['order_count_log'] = order_count_by_county_log['order_count'].apply(lambda x: math.log(x))
plot_graph_of_california(order_count_by_county_log, color_attribute='order_count_log', title='Anzahl an Bestellungen nach County (logarithmisch)', number=5, 
    color_label='Bestellhäufigkeit (logarithmisch)', color_descrete_sequence=px.colors.sequential.Blues, opacity=0.4, hover_data=['order_count'])

## Untersuchung auf besondere Produkte

In [ ]:
products_by_reorder_rate = df_extended.groupby('product_name')['reordered'].agg(['mean', 'count']).sort_values(by='mean',ascending=False).reset_index().query('count > 1000')
products_by_reorder_rate['mean'] = products_by_reorder_rate['mean'].round(2)
fig = px.bar(products_by_reorder_rate.head(10), x='product_name', y='mean',
        title="Top 10 Produkte nach Wiederbestellrate<br><sup>Anteil an Bestellungen, die Wiederbestellungen sind. Filter nach Produkten, welche in mindestens 1000 Bestellungen enthalten sind</sup>", 
        #text="Wiederbestellrate: "+ products_by_reorder_rate.head(10)['mean'].astype(str) + "<br>Bestellungen: " + products_by_reorder_rate.head(10)['count'].astype(str))
        text="Bestellungen:<br>" + products_by_reorder_rate.head(10)['count'].astype(str))
fig.update_xaxes(title='Produkt')
fig.update_yaxes(title='Wiederbestellrate')
fig.update_traces(textposition='inside', textangle=0)
fig.add_annotation(
    showarrow=False,
    text='Abb. 6',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.8,
    xref='paper',
    yref='paper'
    )
fig.show()

In [ ]:
products_by_reorder_probability = df_extended.groupby(['product_name', 'user_id'])['reordered'].max().reset_index().groupby('product_name')['reordered'].mean().sort_values(ascending=False).reset_index()
products_by_reorder_probability['reordered'] = products_by_reorder_probability['reordered'].round(2)

df_temp = df_extended.groupby('product_name').agg({'order_id': 'count', 'user_id': 'nunique'}).reset_index().rename(columns={'order_id': 'order_count'}).query('order_count > 1000')
products_by_reorder_probability = products_by_reorder_probability.merge(df_temp, on='product_name')

#fig = make_subplots(2,2, specs=[[{"type": "bar"}, {"type": "table"}], [{"type": "bar"}, {"type": "table"}]], column_widths=[3,1],horizontal_spacing = 0.01)
fig = make_subplots(2,1, specs=[[{"type": "bar"}], [{'type': "bar"}]], column_widths=[3],horizontal_spacing = 0.01)

fig.add_trace(go.Bar(x=products_by_reorder_probability.head(10)['product_name'], y=products_by_reorder_probability.head(10)['reordered'], 
                    text="Wiederbestell-<br>rate: " + products_by_reorder_probability.head(10)['reordered'].astype(str) + "<br>Bestellungen: <br>" + products_by_reorder_probability.head(10)['order_count'].astype(str) + "<br>individuelle<br>Kunden: <br>" + products_by_reorder_probability.head(10)['user_id'].astype(str), 
                    textposition='inside',
                    textangle=0,
                    marker_color='rgba(0,150,0,0.8)',
                    ), row=1, col=1)
fig.update_xaxes(title='')
fig.update_yaxes(title='Wiederbestellrate')
# fig.add_trace(go.Table(
#                         header=dict(values=['Produkt', 'Wiederbestellwahrscheinlichkeit', 'Bestellhäufigkeit'], align='left'),
#                         cells=dict(
#                                 values=[products_by_reorder_probability.head(10)['product_name'],
#                                     products_by_reorder_probability.head(10)['reordered'], 
#                                     products_by_reorder_probability.head(10)['order_count']],
#                                 align='left'
#                         ),
#                         columnwidth=[2,1,1]
#                     ), row=1, col=2)

fig.add_trace(go.Bar(x=products_by_reorder_probability.tail(10)['product_name'], y=products_by_reorder_probability.tail(10)['reordered'], 
                    text="Wiederbestell-<br>rate: " + products_by_reorder_probability.tail(10)['reordered'].astype(str) + "<br>Bestellungen: <br>" + products_by_reorder_probability.tail(10)['order_count'].astype(str) + "<br>individuelle<br>Kunden: <br>" + products_by_reorder_probability.tail(10)['user_id'].astype(str), 
                    textposition='inside',
                    textangle=0,
                    marker_color='rgba(150,0,0,0.8)',
                    ), row=2, col=1)
# fig.add_trace(go.Table(
#                         header=dict(values=['Produkt', 'Wiederbestellwahrscheinlichkeit', 'Bestellhäufigkeit'], align='left'), 
#                         cells=dict(
#                             values=[products_by_reorder_probability.tail(10)['product_name'],
#                                 products_by_reorder_probability.tail(10)['reordered'],
#                                 products_by_reorder_probability.tail(10)['order_count']],
#                             align='left'
#                         ),
#                         columnwidth=[2,1,1],
#                     ), row=2, col=2)

fig.update_layout(title= dict(text="Top und Flop 10 Produkte nach Wiederbestellwahrscheinlichkeit<br><sup>Anteil an Kunden, die mindestens ein Mal wiederbestellt haben. Filter nach Produkten, welche in mindestens 1000 Bestellungen enthalten sind</sup>", font_size=17, xanchor= "left", yanchor= "top"), height=1100, margin=dict(l=75, r=30, t=100, b=0), showlegend=False)
fig.add_annotation(
    showarrow=False,
    text='Abb. 7',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.15,
    xref='paper',
    yref='paper'
    )
fig.show()

Es sollte untersucht werden, ob die Flop 10 Produkte Qualitätsmängel aufweisen, oder ob diese pro Person nicht so häufig gebraucht werden. Denn obwohl bei niedriger Anzahl an Bestellungen eines Produktes eine niedrigere Wiederbestellrate erwartet werden kann, falls das Produkt beispielsweise neu im Sortiment wäre, befinden sich in den Top 10 Produkten nach Wiederbestellrate Produkte, die ähnliche Bestellmengen aufweisen wie die Flop 10 Produkte. Dies könnte darauf hindeuten, dass die Flop 10 Produkte Qualitätsmängel aufweisen. Die Produkte mit hoher Wiederbestellrate deuten darauf hin, dass die Kunden mit diesen Produkten sehr zufrieden sind. Da von diesen Top 10 Produkten 7 Produkte unter 4000 Bestellungen haben, sollte für diese Produkte mehr Werbung geschalten oder Kunden im Shop diese Produkte vorgeschlagen werden, um somit die Kundenbindung und Absatzzahlen zu erhöhen.

In [ ]:
# == 1 jenachdem, ob die häufigkeit sich nur auf wiederbestellte Bestellungen bezieht
df_temp = df_extended[df_extended.reordered == 1]
products_by_average_order_count = df_temp.groupby('product_name')['reordered'].agg(['sum', 'count']).reset_index().query('count > 500')
products_by_average_order_count = products_by_average_order_count.merge(df_temp.groupby('product_name')['user_id'].nunique().reset_index().rename(columns={'user_id': 'unique_user_count'}), on='product_name')
products_by_average_order_count['mean'] = products_by_average_order_count['sum'] / products_by_average_order_count['unique_user_count']
products_by_average_order_count['mean'] = products_by_average_order_count['mean'].round(2)

order_and_user_count_by_product = df_extended.groupby('product_name').agg({'order_id': 'nunique', 'user_id': 'nunique'}).reset_index().rename(columns={'order_id': 'order_count', 'user_id': 'user_count'})

products_by_average_order_count = products_by_average_order_count.merge(order_and_user_count_by_product, on='product_name')
products_by_average_order_count.sort_values(by='mean',ascending=False, inplace=True)
#fig = px.bar(products_by_average_order_count.sort_values(by='mean',ascending=False).head(10), x='product_name', y='mean',title="Top 10 Produkte nach durchschnittlicher Wiederbestellhäufigkeit pro User", text='mean', hover_data=['count'])

fig = go.Figure()
fig.add_trace(go.Bar(x=products_by_average_order_count.head(10)['product_name'], 
                    y=products_by_average_order_count.head(10)['mean'],
                    text="Wiederbestell-<br>häufigkeit: " + products_by_average_order_count.head(10)['mean'].astype(str) + "<br>Bestellungen: <br>" + products_by_average_order_count.head(10)['order_count'].astype(str) + "<br>individuelle<br>Kunden: <br>" + products_by_average_order_count.head(10)['user_count'].astype(str),
                    ))
fig.update_xaxes(title='Produkt')
fig.update_yaxes(title='Bestellhäufigkeit pro User')
fig.update_layout(title= dict(text="""Top 10 Produkte nach durchschnittlicher Wiederbestellhäufigkeit pro User<br><sup>Durchschnittliche Wiederbestellhäufigkeit von Kunden, welche das Produkt mindestens ein Mal wiederbestellt haben. Filter nach Produkten, die mindestens 500 Mal wiederbestellt wurden</sup>
                                        """, 
                font_size=17, xanchor= "left", yanchor= "top"), height=600, margin=dict(l=75, r=30, t=100, b=50), showlegend=False)
fig.add_annotation(
    showarrow=False,
    text='Abb. 8',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.5,
    xref='paper',
    yref='paper'
    )
fig.show()

Nach dieser Kennzahl lassen sich Produkte finden, welche eine starke Kundenbindung aufweisen. Auffällig ist, dass aus den Top 10 Produkten nach dieser Kennzahl 9 Produkte Milch sind. Dies ist durch die kurze Haltbarkeit von diesen zu erklären, weswegen diese Produkte häufiger bestellt werden, da sie schnell verbraucht und nicht lange gelagert werden können. Foglich lässt sich daraus nicht schließen, dass diese Produkte tatsächlich die stärkste Kundenbindung aufweisen. Jedoch lässt sich eine Unzufriedenheit der Kunden mit diesem Produkt mit hoher Wahrscheinlichkeit ausschließen.

## Fragen der Personalabteilung

### Frage P.1: Untersuchung auf ein Zusammenhang zwischen dem Trinkgeldgeben und dem Vorhandensein eines der folgenden Produkte in einer Bestellung: 24852, 24964, 2120

Um zu überprüfen, ob es einen Zusammenhang zwischen dem Vorhandensein eines der Artikel mit den Artikelnummers 24852, 24964 und 2120 und dem Erhalt von Trinkgeld gibt, werden Hypothesentests durchgeführt, um daraus Handlungsempfehlungen für das Unternehmen zu schließen.

#### Artikel Banana (24852)

+ H₀: Das Erhalten von Trinkgeld ist unahängig von dem Vorhandensein des Produktes Banana (24852) in einer  Bestellung
+ H₁: Das Erhalten von Trinkgeld ist ahängig von dem Vorhandensein des Produktes Banana (24852) in einer  Bestellung

Es wird mit einem Signifikanzniveau von α=5% getestet.

In [ ]:
filtered_df = df_extended.copy()
filtered_df['24852_in_Bestellung'] = filtered_df.groupby('order_id')['product_id'].transform(lambda x: 24852 in x.values)
filtered_df['tip'] = filtered_df['tip'].astype(bool)
filtered_df = filtered_df[['order_id','24852_in_Bestellung', 'tip']].drop_duplicates().rename(columns={'tip': 'Trinkgeld'})
pd.crosstab(filtered_df['24852_in_Bestellung'], filtered_df['Trinkgeld'],margins=True)

In [ ]:
res = chi2_contingency(pd.crosstab(filtered_df['24852_in_Bestellung'], filtered_df['Trinkgeld']))
print('p-value: ', res[1])

Da der p Werte von 0.00 unter dem Signifikanzniveau von 5% liegt, wird H₀ verworfen und H₁ angenommen.
Das Erhalten von Trinkgeld ist daher signifikant abhängig von dem Vorhandensein des Produktes Banana (24852) in einer  Bestellung.

#### Artikel Organic Garlic (24964)

+ H₀: Das Erhalten von Trinkgeld ist unahängig von dem Vorhandensein des Produktes Organic Garlic (24964) in einer  Bestellung
+ H₁: Das Erhalten von Trinkgeld ist ahängig von dem Vorhandensein des Produktes Organic Garlic (24964) in einer  Bestellung

Es wird mit einem Signifikanzniveau von α=5% getestet.

In [ ]:
filtered_df = df_extended.copy()
filtered_df['24964_in_Bestellung'] = filtered_df.groupby('order_id')['product_id'].transform(lambda x: 24964 in x.values)
filtered_df['tip'] = filtered_df['tip'].astype(bool)
filtered_df = filtered_df[['order_id','24964_in_Bestellung', 'tip']].drop_duplicates().rename(columns={'tip': 'Trinkgeld'})
pd.crosstab(filtered_df['24964_in_Bestellung'], filtered_df['Trinkgeld'],margins=True)

In [ ]:
res = chi2_contingency(pd.crosstab(filtered_df['24964_in_Bestellung'], filtered_df['Trinkgeld']))
print('p-value: ', res[1])

Da der p Werte von 0.28 nicht unter dem Signifikanzniveau von 5% liegt, kann H₀ nicht verworfen werden.
Das Erhalten von Trinkgeld ist daher nicht signifikant abhängig von dem Vorhandensein des Produktes Organic Garlic (24964) in einer  Bestellung.

#### Artikel Sauvignon Blanc (2120)

+ H₀: Das Erhalten von Trinkgeld ist unahängig von dem Vorhandensein des Produktes Sauvignon Blanc (2120) in einer  Bestellung
+ H₁: Das Erhalten von Trinkgeld ist ahängig von dem Vorhandensein des Produktes Sauvignon Blanc (2120) in einer  Bestellung

Es wird mit einem Signifikanzniveau von α=5% getestet.

In [ ]:
filtered_df = df_extended.copy()
filtered_df['2120_in_Bestellung'] = filtered_df.groupby('order_id')['product_id'].transform(lambda x: 24852 in x.values)
filtered_df['tip'] = filtered_df['tip'].astype(bool)
filtered_df = filtered_df[['order_id','2120_in_Bestellung', 'tip']].drop_duplicates().rename(columns={'tip': 'Trinkgeld'})
pd.crosstab(filtered_df['2120_in_Bestellung'], filtered_df['Trinkgeld'],margins=True)

In [ ]:
res = chi2_contingency(pd.crosstab(filtered_df['2120_in_Bestellung'], filtered_df['Trinkgeld']))
print('p-value: ', res[1])

Da der p Werte von 0.0 unter dem Signifikanzniveau von 5% liegt, wird H₀ verworfen und H₁ angenommen.
Das Erhalten von Trinkgeld ist daher signifikant abhängig von dem Vorhandensein des Produktes Sauvignon Blanc (2120) in einer  Bestellung.

Die Produkte Banana (24852) und Sauvignon Blanc (2120) sind also signifikant mit dem Erhalt von Trinkgeld verbunden. Das Produkt Organic Garlic (24964) ist nicht signifikant mit dem Erhalt von Trinkgeld verbunden. Nun werden diese Produkte mit ihren jeweiligen Kategorien verglichen, um herauszufinden, ob die speziellen Produke im Kausalzusammenhang mit der erhöhten relativen Trinkgeldhäufigkeit stehen, oder ob die Produkte im normalen Trend ihrer Kategorien liegen. Daraus soll gefolgert werden können, ob die Verteilung der Kategorien auf die Fahrer auf eine bestimmte Weise gestaltet werden soll, um fairere Bedingungen für alle Fahrer zu schaffen.

In [ ]:
tip_prob_by_product_id = df.groupby(['product_id'])['tip'].agg(['count', 'mean']).reset_index().query('count > 200').sort_values(by='mean', ascending=False)
tip_prob_by_product_id = tip_prob_by_product_id.merge(df[['product_id', 'department', 'product_name']].drop_duplicates(), on='product_id')
highlighted_points = tip_prob_by_product_id.query('product_id in [24852, 24964, 2120]')
global_average_tip_percentage = df_extended.groupby('order_id')['tip'].first().mean()

fig = px.box(tip_prob_by_product_id, x='department', y='mean', title='Relative Trinkgeldhäufigkeit der in einer Kategorie sich befindenden Produkte<br><sup>Filter Nach Produkten, welche in mindestens 200 Bestellungen enthalten sind</sup>', hover_data=['product_id', 'count', 'product_name'])
fig.add_hline(y=global_average_tip_percentage, line_dash="dash", line_color="orange", annotation_text="<b>durchschnittliche relative Trinkgeldhäufigkeit <br> aller Bestellungen</b>", annotation_position="bottom left", annotation_font_size=12, annotation_font_color='orange')
for i,product_id in enumerate(highlighted_points['product_id'].unique().tolist()):
    product_name = highlighted_points.query('product_id == @product_id')['product_name'].unique()[0]
    fig.add_trace(go.Scatter(x=highlighted_points.query('product_id == @product_id')['department'], y=highlighted_points.query('product_id == @product_id')['mean'], hoverinfo='text', mode='markers', marker=dict(size=10, color=i, line=dict(width=2,color='DarkSlateGrey')), name=f'{product_name}: {product_id}'))
fig.update_xaxes(title='Kategorie')
fig.update_yaxes(title='relative Trinkgeldhäufigkeit', rangemode='tozero')
fig.add_annotation(
    showarrow=False,
    text='Abb. 9',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.3,
    xref='paper',
    yref='paper'
    )
fig.show()

Aus den zu untersuchenden Produkten liegen die Produkte Sauvignon Blanc (2120) und Organic Garlic (24964) im Trend ihrer jeweiligen Kategorien im Hinblick auf die relative Trinkgeldhäufigkeit. Das Produkt Banana (24852) liegt jedoch deutlich über dem Trend von dessen Kategorie.

### Frage P.2: Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

Vergleich der Counties im Hinblick auf die relative Trinkgeldhäufigkeit aller Bestellungen

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
order_id_tip_info = df_extended[['order_id', 'tip', 'county']].drop_duplicates()
tip_prob_by_county = order_id_tip_info.groupby(['county'])['tip'].agg(['count', 'mean']).reset_index().sort_values(by='mean', ascending=False).rename(columns={'mean':'Trinkgeldrate', 'count':'Bestellungen'})

fig2 = px.scatter(tip_prob_by_county, x='county', y='Trinkgeldrate', title='Relative Trinkgeldhäufigkeit pro County', hover_data=['Bestellungen'], size='Bestellungen', size_max=20)
fig2.add_hline(y=global_average_tip_percentage, line_dash="dash", line_color="orange", annotation_text="<b>durchschnittliche relative Trinkgeldhäufigkeit <br> aller Bestellungen</b>", annotation_position="bottom left", annotation_font_size=12, annotation_font_color='orange')
fig2.update_traces(marker_sizemin = 2)
fig.update_xaxes(title='County')
fig.update_yaxes(title='relative Trinkgeldhäufigkeit', rangemode='tozero')

fig.add_trace(fig2.data[0])
#fig.add_trace(go.Scatter(x=tip_prob_by_county['county'], y=tip_prob_by_county['count'], mode='markers', marker={'size':10}, name='Anzahl Bestellungen'), secondary_y=True)
fig.update_layout(title= dict(text="Relative Trinkgeldhäufigkeit pro County<br><sup>Die Größe der Datenpunkte beschreibt die Anzahl an Bestellungen pro County</sup>", font_size=17, xanchor= "left", yanchor= "top"))
fig.add_hline(y=global_average_tip_percentage, line_dash="dash", line_color="orange", annotation_text="<b>durchschnittliche relative Trinkgeldhäufigkeit <br> aller Bestellungen</b>", annotation_position="bottom left", annotation_font_size=12, annotation_font_color='orange')
fig.add_annotation(
    showarrow=False,
    text='Abb. 10',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.5,
    xref='paper',
    yref='paper'
    )
fig.show()

plot_graph_of_california(tip_prob_by_county, color_attribute='Trinkgeldrate', title='Relative Trinkgeldhäufigkeit pro County', color_label='relative Trinkgeldhäufigkeit<br>des Counties', hover_data=['Bestellungen'], opacity=0.5, number=11)

Auf County Ebene können regionale Unterschiede im Hinblick auf die Bereitschaft dazu, Trinkgeld zu geben, erkannt werden. Auf ganz Kalifornien bezogen lassen sich jedoch nur wenige Cluster an Counties finden, die ähnliches Verhalten beim Trinkgeld Geben aufweisen. Erkennen lässt sich jedoch, dass die Counties an der Westküste von Mittelkalifornien im Hinblick auf die relative Trinkgeldhäufigkeit großteils unter dem Durchschnitt von Kalifornien liegen. Stark über dem Durchschnitt von Kalifornien liegen die Counties San Diego, Santa Barbara, Lassen und Del Norte, welche alle eine relative Trinkgeldhäufigkeit von über 50% aufweisen. Stark unter dem Durchschnitt sind Trinity, Tulare und Inyo. Jedoch gibt es in all diesen Counties im Countyvergleich wenig Bestellungen, wodurch, insbesondere bei Trinity mit nur 185 individuellen Bestellungen, die statistische Aussagekraft nicht so hoch ist. Aus diesem Grund sollte diese Analyse in Zukunft nochmals durchgeführt werden, um eine bessere Aussagekraft zu erhalten.

### Frage P.3: Welche Attribute einer Bestellung wirken sich auf das Trinkgeldverhalten aus?

Wie bereits ermittelt, hat die Kategorie und das County einen Einfluss auf die relative Trinkgeldhäufigkeit. Besonders die Kategorie "Alcohol" sticht dabei positiv heraus.
Es wird nun ermittelt, welche andere Attribute einer Bestellung Einfluss auf die relative Trinkgeldhäufigkeit haben.

#### Bestellgröße

In [ ]:
order_size_by_order_id = df_extended.groupby('order_id')['tip'].agg(['count', 'first']).reset_index().rename(columns={'count':'order_size', 'first':'tip'})
bins = [0, 5, 10, 15, 20, 30, 50, 70, 100, 150]
order_size_by_order_id['order_size_bin'] = pd.cut(order_size_by_order_id['order_size'], bins=bins)
tip_prob_by_order_size = order_size_by_order_id.groupby('order_size_bin')['tip'].agg(['count', 'mean']).reset_index().sort_values(by='order_size_bin')
tip_prob_by_order_size['order_size_bin'] = tip_prob_by_order_size['order_size_bin'].astype(str)

fig = make_subplots(rows=2,cols=1, specs=[[dict(type= "bar")],[dict(type= "table")]])

fig.add_trace(go.Bar(x=tip_prob_by_order_size['order_size_bin'], y=tip_prob_by_order_size['mean'], 
                    text="Trinkgeldrate: " + tip_prob_by_order_size["mean"].round(3).astype(str) + "<br>Bestellungen: " + tip_prob_by_order_size["count"].astype(str),
                    #textfont=dict(size=10),
                    textposition='inside',
                    textangle=0,
                    ), row=1, col=1)
fig.update_xaxes(title='Bestellgröße')
fig.update_yaxes(title='relative Trinkgeldhäufigkeit')

fig.add_trace(go.Table(header=dict(values=['Bestellgröße', 'Anzahl Bestellungen', 'relative Trinkgeldhäufigkeit'],
                            align='left',
                            font= dict(size=15)),
                        cells=dict(values=[tip_prob_by_order_size.order_size_bin, tip_prob_by_order_size['count'], tip_prob_by_order_size['mean'].round(3)],
                            align='left',
                            font= dict(size=14),
                            height=29),
                        columnwidth=[100, 100, 100]
                    ), row=2, col=1)

fig.update_layout(title= dict(text="Relative Trinkgeldhäufigkeit pro Anzahl an verschiedenen Produkten in einer Bestellung", font_size=17, xanchor= "left", yanchor= "top"), height=800, margin=dict(l=75, r=20, t=100, b=0))
fig.add_annotation(
    showarrow=False,
    text='Abb. 12',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=0.02,
    xref='paper',
    yref='paper'
    )
fig.show()

Anhand der Daten zu den Bestellungen lässt sich erkennen, dass es pro Bestellung eine höhere relative Trinkgeldhäufigkeit gibt, je mehr verschiedene Positionen in der Bestellung vertreten sind. Bei den Bestellungen mit sehr vielen Positionen ist die Datengrundlage jedoch sehr dünn, weswegen die statistische Aussagekraft hier geringer ausfällt als bei den anderen Bestellgrößen. 

#### Wochentag und Tage seit der letzten Bestellung

In [ ]:
tip_percentage_by_weekday = df_extended.groupby(['order_dow', 'order_id'])['tip'].first().reset_index()
tip_percentage_by_weekday = tip_percentage_by_weekday.groupby('order_dow')['tip'].agg(['count', 'mean']).reset_index().rename(columns={'mean':'tip'})
tip_percentage_by_weekday['weekday'] = tip_percentage_by_weekday['order_dow'].map({0:'Montag', 1:'Dienstag', 2:'Mittwoch', 3:'Donnerstag', 4:'Freitag', 5:'Samstag', 6:'Sonntag'})

fig = make_subplots(rows=1,cols=2, specs=[[dict(type= "bar"),dict(type= "table")]], column_widths=[2, 1], horizontal_spacing=0.01)
fig.add_trace(go.Bar(x=tip_percentage_by_weekday['weekday'], y=tip_percentage_by_weekday['tip'],
                    text="Trinkgeldrate: " + tip_percentage_by_weekday["tip"].round(3).astype(str) + "<br>Bestellungen: " + tip_percentage_by_weekday["count"].astype(str),
                    #textfont=dict(size=10),
                    textposition='inside',
                    textangle=0,
                    hovertext=tip_percentage_by_weekday['count']
                    ), row=1, col=1)
fig.update_xaxes(title='Wochentag')
fig.update_yaxes(title='relative Trinkgeldhäufigkeit')

fig.add_trace(go.Table(header=dict(values=['Wochentag', 'Anzahl Bestellungen', 'relative Trinkgeldhäufigkeit'],
                            align='left',
                            font= dict(size=15)),
                        cells=dict(values=[tip_percentage_by_weekday.weekday, tip_percentage_by_weekday['count'], tip_percentage_by_weekday['tip'].round(3)],
                            align='left',
                            font= dict(size=14),
                            height = 40),
                        columnwidth=[100, 100, 100]
                    ), row=1, col=2)

fig.update_layout(title= dict(text="Relative Trinkgeldhäufigkeit pro Wochentag", font_size=17, xanchor= "left", yanchor= "top"), height=500, margin=dict(l=75, r=20, t=100, b=30))
fig.add_annotation(
    showarrow=False,
    text='Abb. 13',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.1,
    xref='paper',
    yref='paper'
    )
fig.show()



Montags und Dienstag sind sowohl relative Trinkgeldhäufigkeit und die Anzahl an Bestellungen am Höchsten. Eine mögliche Erklärung hierfür ist, dass die wöchentlichen Einkäufe am Montag und Dienstag stattfinden, bei welchen die Trinkgeldrate höher sein könnte. Diese Hypothese wird nun überprüft.

In [ ]:
dspo_of_order = df_extended.groupby('order_id').agg({'tip': 'first', 'days_since_prior_order': 'first'}).reset_index()
tip_prob_by_dspo = dspo_of_order.groupby('days_since_prior_order')['tip'].agg(['count', 'mean']).reset_index().rename(columns={'mean':'Trinkgeldrate', 'count':'Anzahl Bestellungen'})
tip_prob_by_dspo['days_since_prior_order'] = tip_prob_by_dspo['days_since_prior_order'].astype(int)

fig = make_subplots(rows=2,cols=1, specs=[[{'type': 'bar'}], [{'type': 'bar'}]],
    vertical_spacing=0.12,
    subplot_titles=("Relative Trinkgeldhäufigkeit pro Tage seit letzter Bestellung<br><sup>Die Größe der Datenpunkte beschreibt die Anzahl an Bestellungen pro Anzahl vergangener Tage</sup>", "Anzahl an wöchentlichen Bestellungen pro Wochentag"))

fig2 = px.scatter(tip_prob_by_dspo, x='days_since_prior_order', y='Trinkgeldrate', size='Anzahl Bestellungen', size_max=35)
fig.add_trace(fig2.data[0], row=1, col=1)

weekday_of_weekly_order = df_extended.query('days_since_prior_order == 7').groupby('order_dow')['order_id'].nunique().reset_index()
weekday_of_weekly_order['weekday'] = weekday_of_weekly_order['order_dow'].map({0:'Montag', 1:'Dienstag', 2:'Mittwoch', 3:'Donnerstag', 4:'Freitag', 5:'Samstag', 6:'Sonntag'})
fig.add_trace(go.Bar(x=weekday_of_weekly_order['weekday'], y=weekday_of_weekly_order['order_id'], showlegend=False, text=weekday_of_weekly_order['order_id']), row=2, col=1)
#fig.add_trace(go.Bar(x=tip_prob_by_dspo['days_since_prior_order'], y=tip_prob_by_dspo['Anzahl Bestellungen'], name='Anzahl Bestellungen'), row=2, col=1)

fig.update_xaxes(title='Tage seit letzter Bestellung', row=1, col=1)
fig.update_yaxes(title='relative Trinkgeldhäufigkeit', rangemode='tozero', row=1, col=1)
fig.update_xaxes(title='Wochentag', row=2, col=1)
fig.update_yaxes(title='Anzahl Bestellungen', row=2, col=1)

fig.update_layout(title= dict(text="Analyse des Grundes für den Zusammenhang zwischen dem Wochentag und der relativen Trinkgeldhäufigkeit", font_size=17, xanchor= "left", yanchor= "top", y=0.96), height=900, margin=dict(l=75, r=20, t=100, b=50))
fig.add_annotation(
    showarrow=False,
    text='Abb. 14',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.02,
    xref='paper',
    yref='paper'
    )
fig.show()

Die relative Trinkgeldhäufigkeit ist bei den wöchentlichen Bestellungen am Höchsten, welche am Häufigsten an einem Montag oder Dienstag stattfinden. Um herauszufinden, ob dies den Zusammenhang zwischen Wochentag und relativer Trinkgeldhäufigkeit erklärt, werden nun die wöchentlichen Bestellungen aus den Daten gefiltert und die relative Trinkgeldhäufigkeit pro Wochentag erneut ermittelt.

In [ ]:
tip_percentage_by_weekday = df_extended.query('days_since_prior_order != 7').groupby(['order_dow', 'order_id'])['tip'].first().reset_index()
tip_percentage_by_weekday = tip_percentage_by_weekday.groupby('order_dow')['tip'].agg(['count', 'mean']).reset_index().rename(columns={'mean':'tip'})
tip_percentage_by_weekday['weekday'] = tip_percentage_by_weekday['order_dow'].map({0:'Montag', 1:'Dienstag', 2:'Mittwoch', 3:'Donnerstag', 4:'Freitag', 5:'Samstag', 6:'Sonntag'})

fig = make_subplots(rows=1,cols=2, specs=[[dict(type= "bar"),dict(type= "table")]], column_widths=[2, 1], horizontal_spacing=0.01)
fig.add_trace(go.Bar(x=tip_percentage_by_weekday['weekday'], y=tip_percentage_by_weekday['tip'],
                    text="Trinkgeldrate: " + tip_percentage_by_weekday["tip"].round(3).astype(str) + "<br>Bestellungen: " + tip_percentage_by_weekday["count"].astype(str),
                    #textfont=dict(size=10),
                    textposition='inside',
                    textangle=0,
                    hovertext=tip_percentage_by_weekday['count']), row=1, col=1)
        
fig.update_xaxes(title='Wochentag')
fig.update_yaxes(title='relative Trinkgeldhäufigkeit')

fig.add_trace(go.Table(header=dict(values=['Wochentag', 'Anzahl Bestellungen', 'relative Trinkgeldhäufigkeit'],
                            align='left',
                            font= dict(size=15)),
                        cells=dict(values=[tip_percentage_by_weekday.weekday, tip_percentage_by_weekday['count'], tip_percentage_by_weekday['tip'].round(3)],
                            align='left',
                            font= dict(size=14), height = 40),
                        columnwidth=[100, 100, 100]
                    ), row=1, col=2)

fig.update_layout(title= dict(text="Relative Trinkgeldhäufigkeit pro Wochentag ohne wöchentliche Bestellungen", font_size=17, xanchor= "left", yanchor= "top"), height=500, margin=dict(l=75, r=20, t=100, b=30))
fig.add_annotation(
    showarrow=False,
    text='Abb. 15',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.1,
    xref='paper',
    yref='paper'
    )
fig.show()

Die wöchentlichen Bestellungen, welche eine besonders hohe relative Trinkgeldhäufigkeit aufweisen, fallen zwar am Häufigsten auf einen Montag oder Dienstag, was die erhöhte durchschnittliche Trinkgeldhäufigkeit jedoch nur bedingt erklären kann. Werden die wöchentlichen Bestellungen herausgefiltert, so liegt die relative Trinkgeldhäufigkeit am Montag und Dienstag zwar niedriger als zuvor, aber immer noch deutlich über den den anderen Tagen. Folglich ist die erhöhte relative Trinkgeldhäufigkeit am Montag und Dienstag nicht nur auf die wöchentlichen Bestellungen zurückzuführen.

#### Bestellzeit

In [ ]:
tip_prob_by_time = df_extended.groupby('order_hour_of_day').agg({'order_id': 'nunique', 'tip': 'mean'}).reset_index().rename(columns={'tip':'Trinkgeldrate', 'order_id':'Anzahl Bestellungen'})

fig = go.Figure()
fig2 = px.scatter(tip_prob_by_time, x='order_hour_of_day', y='Trinkgeldrate', size='Anzahl Bestellungen', size_max=35)
fig.add_trace(fig2.data[0])

fig.update_xaxes(title='Uhrzeit')
fig.update_yaxes(title='Relative Trinkgeldhäufigkeit', rangemode="tozero")

fig.update_layout(title= dict(text="Relative Trinkgeldhäufigkeit pro Uhrzeit<br><sup>Die Größe der Datenpunkte beschreibt die Anzahl an Bestellungen pro Tagesstunde</sup>", font_size=17, xanchor= "left", yanchor= "top"), height=500, margin=dict(l=75, r=20, t=100, b=30))
fig.add_annotation(
    showarrow=False,
    text='Abb. 16',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.1,
    xref='paper',
    yref='paper'
    )
fig.show()

Es lässt sich erkennen, dass die relative Trinkgelfhäufigkeit morgens bis spätnachmittags niedriger ist als am Abend und in der Nacht. Abends und Nachts scheint die relative Tringeldhäufigkeit auf einem ungefähr gleichen Niveau zu sein, während morgens bis spätnachmittags das Niveau ebenfalls sehr ähnlich ist. Deshalb wird im Folgenden ein Hypothesentest durchgeführt, ob die relative Trinkgeldhäufigkeit am Abend und in der Nacht statistisch signifikant höher ist als morgens bis spätnachmittags.

+ H₀: Die relative Trinkgeldhäufigkeit am Abend und in der Nacht ist nicht statistisch signifikant höher als morgens bis spätnachmittags.
+ H₁: Die relative Trinkgeldhäufigkeit am Abend und in der Nacht ist statistisch signifikant höher als morgens bis spätnachmittags.

Es wird mit einem Signifikanzniveau von α=5% getestet.

In [ ]:
df_night = df_extended.copy()
df_night['19-4Uhr'] = df_night['order_hour_of_day'].apply(lambda x: True if x >= 19 or x <= 4 else False)
df_night['tip'] = df_night['tip'].astype(bool)
df_night_reduced = df_night[['order_id', '19-4Uhr', 'tip']].drop_duplicates().rename(columns={'tip':'Trinkgeld'})
display(pd.crosstab(df_night_reduced['19-4Uhr'], df_night_reduced['Trinkgeld']))
res = chi2_contingency(pd.crosstab(df_night_reduced['19-4Uhr'], df_night_reduced['Trinkgeld']))
print('p-Wert: ', res[1])


Der p-Wert von 0.00 liegt unter dem Signifikanzniveau von 5%, weswegen die Nullhypothese verworfen wird und die Alternativhypothese angenommen wird. Die relative Trinkgeldhäufigkeit am Abend und in der Nacht ist statistisch signifikant höher als morgens bis spätnachmittags.

#### Regalebene

In [ ]:
tip_percentage_by_isle = df_extended.groupby(['aisle', 'order_id'])['tip'].first().reset_index()
tip_percentage_by_isle = tip_percentage_by_isle.groupby('aisle')['tip'].agg(['count', 'mean']).reset_index().rename(columns={'mean':'tip_percentage'})
tip_percentage_by_isle['department'] = tip_percentage_by_isle['aisle'].map(df_extended.groupby('aisle')['department'].first().to_dict())
tip_percentage_by_isle.sort_values(by='tip_percentage', ascending=False, inplace=True)
fig = px.scatter(tip_percentage_by_isle, x='aisle', y='tip_percentage', size='count', size_max=30, hover_data=['count', 'department'], title='Relative Trinkgeldhäufigkeit pro Regal<br><sub>Die Größe der Datenpunkte beschreibt die Anzahl an Bestellungen des Regals</sub>')
fig.update_xaxes(title='Regal')
fig.update_yaxes(title='Relative Trinkgeldhäufigkeit', rangemode="tozero")
fig.add_annotation(
    showarrow=False,
    text='Abb. 17',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-1.6,
    xref='paper',
    yref='paper'
    )
fig.show()

Da bereits analysiert wurde, dass die Kategorie 'Alcohol' die größte relative Trinkgeldhäufigkeit aufweist, ist es nicht verwunderlich, dass die Regale, in denen die Produkte aus dieser Kategorie stehen, ebenfalls eine hohe relative Trinkgeldhäufigkeit aufweisen.

#### Bestellnummer des Users

In [ ]:
order_count_by_order_number = df_extended.groupby(['order_number', 'order_id'])['tip'].first().reset_index()
bins = [0, 5, 10, 15, 20, 30, 50, 70, 100]
order_count_by_order_number['order_number_bin'] = pd.cut(order_count_by_order_number['order_number'], bins=bins)
tip_prob_by_order_number = order_count_by_order_number.groupby('order_number_bin')['tip'].agg(['count', 'mean']).reset_index().sort_values(by='order_number_bin')
tip_prob_by_order_number['order_number_bin'] = tip_prob_by_order_number['order_number_bin'].astype(str)
#order_count_by_order_number['order_number_bin'] = order_count_by_order_number['order_number_bin'].str.replace('(','').str.replace(']','').str.replace(',',' -')
fig = make_subplots(rows=1,cols=2, specs=[[dict(type= "bar"),dict(type= "table")]], column_widths=[2, 1], horizontal_spacing=0.01)
fig.add_trace(go.Bar(x=tip_prob_by_order_number['order_number_bin'], y=tip_prob_by_order_number['mean'],
            text="Trinkgeldhfk: " + tip_prob_by_order_number["mean"].round(3).astype(str) + "<br>Bestellungen: " + tip_prob_by_order_number["count"].astype(str),
            textposition='inside',
            textangle=0,
            ))
fig.update_xaxes(title='Bestellnummer des Users')
fig.update_yaxes(title='Relative Trinkgeldhäufigkeit')

fig.add_trace(go.Table(header=dict(values=['Bestellnummer des Users', 'Anzahl Bestellungen', 'relative Trinkgeldhäufigkeit'],
                            align='left',
                            font= dict(size=15)),
                        cells=dict(values=[tip_prob_by_order_number.order_number_bin, tip_prob_by_order_number['count'], tip_prob_by_order_number['mean'].round(3)],
                            align='left',
                            font= dict(size=14), height = 40),
                        columnwidth=[100, 100, 100]
                    ), row=1, col=2)

fig.update_layout(title= dict(text="Relative Trinkgeldhäufigkeit pro Bestellnummer des Users", font_size=17, xanchor= "left", yanchor= "top"), height=530, margin=dict(l=75, r=20, t=100, b=50))
fig.add_annotation(
    showarrow=False,
    text='Abb. 18',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.1,
    xref='paper',
    yref='paper'
    )
fig.show()

Es lässt sich erkennen, dass Nutzer, welche den Lieferservice häufiger genutzt haben, eher dazu neigen, Trinkgeld zu geben, als Nutzer, welche den Lieferservice neu nutzen.
Bei Nutzern, welche den Lieferservice über 50 Mal nutzten, sinkt die relative Trinkgeldhäufigkeit jedoch wieder. Eine sinnvolle Erklärung dafür kann jedoch nicht gefunden werden.

#### Zusammenfassung

Insgesamt lässt sich erkennen, dass die relative Trinkgeldhäufigkeit von den Eigenschaften der Produkte selbst wie Kategorie oder Regal sowie dem County, der Bestellgröße, dem Wochentag, der Anzahl an Tage seit der letzten Bestellung, der Uhrzeit und der Bestellnummer des Users beeinflusst wird. Im nächsten Abschnitt wird nun untersucht, ob der User selbst durch seine Historie die relative Trinkgeldhäufigkeit beeinflusst.

### Frage P.4: Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?

Um herauszufinden, ob die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit spielt, werden zwei ML-Modelle erstellt, welche vorhersagen, ob bei einer Bestellung ein Trinkgeld gegeben wird oder nicht. Eins davon wird zusätzlich die Information darüber verarbeiten, was die relative Trinkgeldhäufigkeit des Users bis zum Zeitpunkt der Bestellung ist. Wenn es die erste Bestellung ist, so ist sie 0. Wenn beide Modelle ähnlich genaue Vorhersagen treffen, ist die Historie nicht relevant. Ist jedoch das Modell mit der zusätzlichen Information präziser, so ist die Historie des Users relevant.

In [ ]:
df_for_model_training = df_extended[['order_id','order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'tip_percentage_untill_this_order', 'department', 'tip']]
df_for_model_training = df_for_model_training.dropna()

X = pd.get_dummies(df_for_model_training, columns=['department'])
X = X.groupby(['order_id']).max()
y = X['tip']
X = X.drop(columns=['tip'])

#### Mit Information über die bisherige relative Trinkgeldhäufigkeit des bestellenden Users.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_lrc = LogisticRegression(penalty= "l2", C=1, solver= "saga", max_iter= 100)
best_lrc.fit(X_train, y_train)

y_pred_train_best_lrc = best_lrc.predict(X_train)
y_pred_test_best_lrc = best_lrc.predict(X_test)

print_evaluation("LogisticRegressionClassifier", X_train, X_test, y_train, y_test, y_pred_train_best_lrc, y_pred_test_best_lrc)

#### Ohne Information über die bisherige relative Trinkgeldhäufigkeit des bestellenden Users.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.drop(columns=['tip_percentage_untill_this_order']), y, test_size=0.2, random_state=42)

best_lrc = LogisticRegression(penalty= "l2", C= 1, solver= "saga", max_iter= 100)
best_lrc.fit(X_train, y_train)

y_pred_train_best_lrc = best_lrc.predict(X_train)
y_pred_test_best_lrc = best_lrc.predict(X_test)

print_evaluation("LogisticRegressionClassifier", X_train, X_test, y_train, y_test, y_pred_train_best_lrc, y_pred_test_best_lrc)

Das Modell, welches über die Information über die bisherige relative Trinkgeldhäufigkeit des Users verfügt, kann genauer vorhersagen, ob der User bei der Bestellung ein Trinkgeld geben wird als das Modell, welches über diese Information nicht verfügt. Daraus lässt sich schließen, dass die Historie eines Users eine Rolle in Hinblick auf das Trinkgeldverhalten spielt.

### Spezifische Handlungsempfehlungen für die Personalabteilung
Um die Fahrten besser zu verteilen, sodass die Chance auf Trinkgeld unter den Fahrern fairer verteilt ist, sollte ein ML-Modell erstellt und optimiert werden, welches die Trinkgeldwahrscheinlichkeit bei einer Bestellung voraussagt. Nach dieser Kennzahl können die Fahrten dann optimal verteilt werden, sodass jeder Fahrer eine ähnliche Chance auf Trinkgeld hat. Ist dies nicht möglich, dann sollte besonders darauf geachtet werden, dass Fahrer ähnlich oft Lieferungen aus der sehr vielversprechenden Kategorie 'Alcohol' sowohl den Kategorien 'snacks', 'babies', 'breakfast' und 'bulk' ausliefern, bei welchen die durchschnittliche relative Trinkgeldhäufigkeit über 50% beträgt. Außerdem sollte vermieden werden, dass Fahrer nur an bestimmten Tagen ausliefern, da die Trinkgeldrate an Montagen und Dienstagen am höchsten ist. Fahrer, die Abends und Nachts ausliefern haben eine höhere Aussicht auf ein Trinkgeld als jene, die tagsüber ausliefern, weswegen im Unternehmen mit den Fahrern besprochen werden sollte, ob sie es als fair empfinden, wenn die Fahrer, die nachts arbeiten, als Ausgleich ein höheres Trinkgeld erhalten.

Eine andere Möglichkeit ist es, die Fahrer, die seltener vielversprechende Bestellungen ausliefern, einen Gehaltsausgleich anzubieten. Wenn das Gehalt der Fahrer, die öfters vielversprechende Bestellungen ausliefern, gleich dem Gehalt der Fahrer, die seltener vielversprechende Bestellungen ausliefern, ist, dann enstehen somit mehr Personalkosten, da das Trinkgeld nun nicht mehr aufgeteilt, sondern auf Kosten des Unternehmens ausgegleichen wird. Folglich könnte in diesem Szenario das Gehalt der Fahrer, die im Hinblick auf Trinkgeld vielversprechende Bestellungen ausliefern, gekürzt werden, um diese Kosten auszugleichen. Dies würde jedoch dazu führen, dass die Fahrer, deren Gehalt gekürzt wurde, von dem Trinkgeld der Kunden abhängig ist, weswegen diese Möglichkeit nicht empfohlen wird.

## Fragen der Disposition

### Frage D.1: Prüfung auf auffällige Muster bei dem Wiederbestellverhalten des Produktes "Bag of Organic Bananas" (Produkt-ID: 13176)

In [ ]:
# Anzahl an Bestellungen abhängig von den vergangenen Tagen seit der letzten Bestellung
reorder_count_by_days_since_prior_order = df_extended[df_extended['product_id'] == 13176].groupby(['days_since_prior_order'])['reordered'].sum()

fig = px.line(reorder_count_by_days_since_prior_order, x=reorder_count_by_days_since_prior_order.index, y=reorder_count_by_days_since_prior_order.values, title='Anzahl an Bestellungen abhängig von den Tagen seit der letzten Bestellung von "Bag of Organic Bananas" (Produkt-ID: 13176)')
fig.update_yaxes(rangemode="tozero", title='Order count')
fig.update_xaxes(title='Tage seit letzter Bestellung')
fig.update_yaxes(title='Anzahl an Bestellungen')
fig.add_annotation(
    showarrow=False,
    text='Abb. 19',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.2,
    xref='paper',
    yref='paper'
    )
fig.show()

Es lässt sich erkennen, dass das Produkt "Bag of Organic Bananas" (Produkt-ID: 24852) am Häufigsten im Wochen- oder Monatsintervall wiederbestellt wird.

### Frage D.2: Spielen die Artikel aus dem Department „produce“ in allen Counties eine gleich große Rolle?

In [ ]:
# calculate the rank of each department per county
rank_of_deparment_in_county = df_extended.groupby(['county', 'department'])['order_id'].count().reset_index().rename(columns={'order_id': 'order_count'})
rank_of_deparment_in_county['rank_in_county'] = rank_of_deparment_in_county.groupby(['county'])['order_count'].rank(ascending=False).astype(int).astype(str)

# calculate the share of sales of the department in the county
rank_of_deparment_in_county = rank_of_deparment_in_county.merge(rank_of_deparment_in_county.groupby(['county'])['order_count'].sum().reset_index().rename(columns={'order_count': 'order_count_in_county'}), on='county')
rank_of_deparment_in_county['order_share_of_department_in_county'] = rank_of_deparment_in_county['order_count'] / rank_of_deparment_in_county['order_count_in_county']

# filter for department produce and plot results
rank_of_department_produce_per_county = rank_of_deparment_in_county[rank_of_deparment_in_county['department'] == 'produce'].sort_values(by='rank_in_county', ascending=True)
rank_of_department_produce_per_county.sort_values(by='order_share_of_department_in_county', ascending=False, inplace=True)

global_share_of_department_produce = rank_of_department_produce_per_county['order_count'].sum() / rank_of_department_produce_per_county['order_count_in_county'].sum()

fig = px.scatter(rank_of_department_produce_per_county, x='county', y='order_share_of_department_in_county', 
                title='Anteil der Kategorie produce pro County<br><sub>Die Größe der Datenpunkte beschreibt die Anzahl an Bestellungen im County</sub>',
                size='order_count', size_max=20, hover_data=['order_count', 'order_count_in_county'], color='rank_in_county', labels={'rank_in_county': 'Rang im County'})

fig.add_hline(y=global_share_of_department_produce, line_dash="dash", line_color="red", annotation_text="Anteil der Kategorie produce in Kalifornien", annotation_position="top right")
#fig.add_annotation(x=56,y=rank_of_department_produce_per_county[rank_of_department_produce_per_county['county'] == 'San Bernardino']['order_share_of_department_in_county'].values[0],text="148 Orders")

fig.update_xaxes(title='County')
fig.update_traces(marker_sizemin = 3)
fig.update_yaxes(title='Anteil der Kategorie produce', rangemode="tozero")
fig.update_layout(xaxis_categoryorder = 'total descending')

fig.add_annotation(
    showarrow=False,
    text='Abb. 20',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.6,
    xref='paper',
    yref='paper'
    )
fig.show()
plot_graph_of_california(df = rank_of_department_produce_per_county, title='Anteil der Kategorie produce pro County', color_attribute='order_share_of_department_in_county', color_label='Anteil der Kategorie produce', hover_data=['order_count', 'order_count_in_county', 'rank_in_county'] ,opacity=0.5, number=21)
#plot_graph_of_california(rank_of_department_produce_per_county, 'rank_in_county', 'Rang der Kategorie produce pro County', color_label='Rang im County', hover_data=['order_count', 'order_count_in_county', 'order_share_of_department_in_county'], number=22)


In allen Counties bis auf Del Norte, Tehama, Sierra und San Bernadino ist die Kategorie produce die am Häufigsten bestellte Kategorie. San Bernadino ist mit nur 148 bestellten Produkten jedoch statistisch überhaupt nicht signifikant. Der Anteil von Produkten aus der Kategorie Produce an den insgesamt bestellten Produkten in den Counties, in denen die Kategorie produce auf dem ersten Platz landet, schwankt von 18.4% in Humboldt bis hin zu 40.2% in Sacramento. Bei dem Großteil der Counties liegt der Anteil der Kategorie produce jedoch zwischen 25% und 35%.

### Frage D.3: Welche Counties sind sich ähnlich in Hinblick auf die jeweiligen „Top 10“-Produkte?

Um die Ähnlichkeit der Top 10 zweier Counties zu finden, werden die zwei Counties nach folgendem Prinzip verglichen:
+ Für jedes Produkt der Top 10 von County 1 wird geschaut, an welchem Platz sich das Produkt in der Top 10 von County 2 befindet und die Differenz zwischen den beiden Plätzen wird aufsummiert.
+ Ist das Produkt von County 1 nicht in der Top 10 von County 2, so wird die Differenz als 10 angenommen und somit 10 addiert.
+ Die Gesamtsumme ist die Kennzahl für die Ähnlichkeit der Top 10 der beiden Counties.

In [ ]:
top_10_products_per_county = df_extended.groupby(['county', 'product_id'])['order_id'].count().reset_index().sort_values(by='order_id', ascending=False).groupby('county').head(10).rename(columns={'order_id': 'order_count'})
top_10_products_per_county = top_10_products_per_county.merge(df_extended[['product_id', 'product_name']].drop_duplicates(), on='product_id')
top_10_products_per_county.sort_values(by=['county', 'order_count'], ascending=False, inplace=True)

def get_difference_between_top_10_of_counties(county_1, county_2):
    """calculates the distance between the top 10 products of two counties based on the index of the product in the top 10 list.
    If a product is not in the top 10 list of the other county, it is counted as 11th place and the distance is 10."""
    distance = 0
    top_10_products_county_1 = top_10_products_per_county[top_10_products_per_county['county'] == county_1]['product_id'].to_list()
    top_10_products_county_2 = top_10_products_per_county[top_10_products_per_county['county'] == county_2]['product_id'].to_list()
    for i in range(10):
        try:
            distance += abs(top_10_products_county_1.index(top_10_products_county_2[i]) - i)
        except ValueError:
            distance += 10
    return distance
    #return len(set(top_10_products_county_1) - set(top_10_products_county_2))

top_10_product_distances = pd.DataFrame(columns=['county_1', 'county_2', 'distance'])

counties = df_extended['county'].unique()
for i in range(len(counties)):
    for j in range(i+1, len(counties)):
        top_10_product_distances.loc[len(top_10_product_distances.index)] = [counties[i], counties[j], get_difference_between_top_10_of_counties(counties[i], counties[j])]

top_10_product_distances.sort_values(by='distance', ascending=True, inplace=True)
top_10_product_distances.query('distance <= 10')

### Räumliche Analyse der Bestellungsdaten für die Disposition

Um den Disponenten, die für die Belieferung der Zentren verantwortlich sind, zu helfen, die Zulieferung der Zentren in Kalifornien zu planen, wird eine interaktive Grafik erstellt, welche es jenen erlaubt, eine Kategorie auszuwählen, wodurch angezeigt wird, in welchen Counties diese Kategorie überdurchschnittlich, durchschnittlich oder unterdurchschnittlich nachgefragt wird. Die Klassifikation erfolgt nach folgendem Prinzip:
+ Anhand des durchschnittlichen Anteils der jeweiligen Kategorie in allen Counties sowie der Standardabweichung dieses Anteiles werden die Klassifikationen durchgeführt. Weicht der Anteil der Kategorie in einem County um mindestens ein Mal der Standardabweichung vom Mittelwert nach oben ab, so wird das County als überdurchschnittlich eingestuft. Weicht der Anteil der Kategorie in einem County um mindestens ein Mal der Standardabweichung vom Mittelwert nach unten ab, so wird das County als unterdurchschnittlich eingestuft. Liegt der Anteil der Kategorie in einem County zwischen dem Mittelwert und dem Mittelwert plus/minus einer Standardabweichung, so wird das County als durchschnittlich eingestuft.

Wollen die Disponenten Informationen über ein bestimmtes County und die darin nachgefragten Kategorien, so können jene in einer Tabelle das County auswählen und erhalten dann eine Übersicht über die Kategorien, die in diesem County überdurchschnittlich, durchschnittlich oder unterdurchschnittlich nachgefragt werden.

In [ ]:
share_of_department_in_county = df_extended.groupby(['county', 'department'])['order_id'].count().reset_index().rename(columns={'order_id': 'order_count'})
share_of_department_in_county['order_count_in_county'] = share_of_department_in_county.groupby('county')['order_count'].transform('sum')
share_of_department_in_county['share_of_department_in_county'] = share_of_department_in_county['order_count'] / share_of_department_in_county['order_count_in_county']
share_of_department_in_county['global_average_share_of_department'] = share_of_department_in_county.groupby('department')['share_of_department_in_county'].transform('mean')
share_of_department_in_county['global_average_order_count'] = share_of_department_in_county.groupby('department')['order_count'].transform('mean')
share_of_department_in_county['standard_deviation_of_share_of_department'] = share_of_department_in_county.groupby('department')['share_of_department_in_county'].transform('std')
share_of_department_in_county['multiple_of_std'] = (share_of_department_in_county['share_of_department_in_county'] - share_of_department_in_county['global_average_share_of_department'])/ share_of_department_in_county['standard_deviation_of_share_of_department']
share_of_department_in_county['performance'] = share_of_department_in_county.apply(lambda x: 'überdurchschnittlich' if x['multiple_of_std'] >= 1 else ('unterdurchschnittlich' if x['multiple_of_std'] <=-1 else "durchschnittlich"), axis=1)

In [ ]:
fig = go.Figure()

colorscales = {
    'unterdurchschnittlich':((0.0, 'red'), (1.0, 'red')),
    'überdurchschnittlich':((0.0, 'green'), (1.0, 'green')),
    'durchschnittlich':((0.0, 'grey'), (1.0, 'grey'))
}


for department in share_of_department_in_county.department.unique():
    for i,performance in enumerate(share_of_department_in_county[share_of_department_in_county.department == department].performance.unique()):
        fig.add_trace(go.Choroplethmapbox(
            geojson=counties_geojson,
            locations=share_of_department_in_county[(share_of_department_in_county.department == department) & (share_of_department_in_county.performance == performance)]['county'],
            z=[i,]*len(share_of_department_in_county[(share_of_department_in_county.department == department) & (share_of_department_in_county.performance == performance)]['performance']),
            featureidkey= "properties.NAME",
            colorscale=colorscales[performance],
            marker_opacity=0.5,
            marker_line_width=0,
            name=performance,
            showlegend=True,
            showscale=False,
            hovertemplate = '<b>county</b>: %{location}<br>' +
                            '<b>global_average_share_of_department</b>: %{customdata[0]:.2f}<br>' +
                            '<b>global_average_order_count</b>: %{customdata[1]:.2f}<br>' +
                            '<b>standard_deviation_of_share_of_department</b>: %{customdata[2]:.2f}<br>' +
                            '<b>multiple_of_std</b>: %{customdata[3]:.2f}<br>' +
                            '<b>share_of_department_in_county</b>: %{customdata[4]:.2f}<br>' +
                            '<b>order_count</b>: %{customdata[5]:.2f}<br>' +
                            '<b>order_count_in_county</b>: %{customdata[6]:.2f}<br>' +
                            '<b>performance</b>: %{customdata[7]}<br>',
            customdata=share_of_department_in_county[(share_of_department_in_county.department == department) & (share_of_department_in_county.performance == performance)][['global_average_share_of_department', 'global_average_order_count', 'standard_deviation_of_share_of_department', 'multiple_of_std', 'share_of_department_in_county', 'order_count', 'order_count_in_county', 'performance']].values,
            visible=True if department == share_of_department_in_county.department.unique()[0] else False
        ))

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        buttons=[dict(label = department,
                    method = 'update',
                    args=[{'visible': np.hstack(np.array([[True]*share_of_department_in_county[share_of_department_in_county.department == dep]['performance'].nunique() if dep == department else [False]*share_of_department_in_county[share_of_department_in_county.department == dep]['performance'].nunique() for dep in share_of_department_in_county.department.unique()], dtype='object'))}]) for department in share_of_department_in_county.department.unique()],
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        xanchor="left",
        y=1.15,
        yanchor="top"
        ),
    ],
    mapbox_style="carto-positron",
    mapbox_zoom=4.3,
    mapbox_center = {"lat": 37.278259, "lon": -119.417931},
    margin={"r":0,"l":0,"b":0},
    title=dict(text='County Vergleich der Performance einer Kategorie'),
    )
fig.add_annotation(
    showarrow=False,
    text='Abb. 22',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.04,
    y=0.05,
    xref='paper',
    yref='paper'
    )
fig.show()

In [ ]:
fig = go.Figure()
#fig = make_subplots(rows=1, cols=2, specs=[[{'type':'table'}, {'type':'sunburst'}]])

for county in share_of_department_in_county.county.unique():
    fig.add_trace(go.Table(
        header=dict(values=['Kategorie', 'Durchschnittlicher Anteil der kategorie in Kalifornien',f'Anteil der Kategorie in {county}', 'Bestellhäufigkeit', f'Gesamtbestellungen in {county}', 'Performance'],
                    #fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[share_of_department_in_county[share_of_department_in_county.county == county]['department'],
                            share_of_department_in_county[share_of_department_in_county.county == county]['global_average_share_of_department'].multiply(100).round(2).astype(str) + '%',
                            share_of_department_in_county[share_of_department_in_county.county == county]['share_of_department_in_county'].multiply(100).round(2).astype(str) + '%',
                            share_of_department_in_county[share_of_department_in_county.county == county]['order_count'],
                            share_of_department_in_county[share_of_department_in_county.county == county]['order_count_in_county'],
                            share_of_department_in_county[share_of_department_in_county.county == county]['performance']],
                    #fill_color='lavender',
                    align='left'),
        name=county,
        visible=True if county == share_of_department_in_county.county.unique()[0] else False
    ),#row=1, col=1
    )
    # fig2 = px.sunburst(share_of_department_in_county[share_of_department_in_county.county == county], 
    #                 path=['county', 'department'], 
    #                 values='order_count', 
    #                 color='performance', 
    #                 color_discrete_map={'durchschnittlich': 'grey', 'unterdurchschnittlich': 'red', 'überdurchschnittlich': 'green'}, 
    #                 title='Performance der Kategorien in den Counties',
    #                 ).update_traces(visible=True if county == share_of_department_in_county.county.unique()[0] else False)
    # fig.add_trace(fig2.data[0], row=1, col=2)

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        buttons=[dict(label = county,
                    method = 'update',
                    args=[{'visible': np.hstack(np.array([[True] if county == county else [False] for county in share_of_department_in_county.county.unique()], dtype='object'))}]) for county in share_of_department_in_county.county.unique()],
        direction="down",
        pad={"r": 10, "t": 0},
        showactive=True,
        xanchor="left",
        y=1.15,
        yanchor="top"
        ),
    ],
    title=dict(text='Performance Übersicht aller Kategorien in einem County'),
    )
fig.add_annotation(
    showarrow=False,
    text='Abb. 23',
    font=dict(size=10),
    xanchor='left',
    yanchor='bottom',
    x=-0.05,
    y=-0.2,
    xref='paper',
    yref='paper'
    )
fig.show()


### Spezifische Handlungsempfehlungen für die Disposition:
Folgende 10 Kategorien werden in folgenden Counties besonders stark nachgefragt und sollten daher in diesen Counties von der Disposition besonders stark beliefert werden:
+ Alcohol in Humboldt mit einem Anteil von ca. 10% an allen Bestellungen in Humboldt vs. 1.1% im Durchschnitt
+ Dairy eggs in Trinity mit einem Anteil von ca. 28% an allen Bestellungen in Trinity vs 16.7% im Durchschnitt
+ Beverages in Yuba mit einem Anteil von ca. 15.5% an allen Bestellungen in Yuba vs. 8.7% im Durchschnitt
+ Snacks in Humboldt mit einem Anteil von ca. 16% an allen Bestellungen in Humboldt vs 9.1% im Durchschnitt
+ Produce in Sacramento mit einem Anteil von ca. 40% an allen Bestellungen in Sacramento vs 28.2% im Durchschnitt
+ Dairy eggs in Tehama mit einem Anteil von ca. 23% an allen Bestellungen in Tehama vs 16.7% im Durchschnitt
+ beverages in Humboldt mit einem Anteil von ca. 14% an allen Bestellungen in Humboldt vs 8.8% im Durchschnitt
+ Snacks in El Dorado mit einem Anteil von ca. 14.4% an allen Bestellungen in El Dorado vs 9.1% im Durchschnitt
+ Beverages in Alameda mit einem Anteil von ca. 13.6% an allen Bestellungen in Alameda vs 8.8% im Durchschnitt
+ Beverages in Napa mit einem Anteil von ca. 13.6% an allen Bestellungen in Napa vs 8.8% im Durchschnitt

Folgende 10 Kategorien werden in folgenden Counties besonders schwach nachgefragt und sollten daher in diesen Counties von der Disposition besonders wenig beliefert werden:
+ Beverages in San Francisco mit einem Anteil von ca. 1.5% an allen Bestellungen in San Francisco vs 8.8% im Durchschnitt
+ Produce in Sierra mit einem Anteil von ca. 17.6% an allen Bestellungen in Sierra vs 28.2% im Durchschnitt
+ Produce in Teheran mit einem Anteil von ca. 17.6% an allen Bestellungen in Teheran vs 28.2% im Durchschnitt
+ Snacks in Lake mit einem Anteil von ca. 3.7% an allen Bestellungen in Lake vs 9.1% im Durchschnitt
+ Frozen in Kern mit einem Anteil von ca. 4.4% an allen Bestellungen in Kern vs 6.8% im Durchschnitt
+ Beverages in Nevada mit einem Anteil von ca. 3.5% an allen Bestellungen in Nevada vs 8.8% im Durchschnitt
+ Beverages in San Joaquin mit einem Anteil von ca. 3.5% an allen Bestellungen in San Joaquin vs 8.8% im Durchschnitt
+ Produce in Humboldt mit einem Anteil von ca. 18.4% an allen Bestellungen in Humboldt vs 28.2% im Durchschnitt
+ Dairy eggs in Humboldt mit einem Anteil von ca. 11.4% an allen Bestellungen in Humboldt vs 16.7% im Durchschnitt
+ Produce in San Diego mit einem Anteil von ca. 19% an allen Bestellungen in San Diego vs 28.2% im Durchschnitt